In [ ]:
#Importing all required library
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense , Flatten , Conv2D , MaxPool2D, Dropout
from keras.optimizers import SGD , Adam
from keras.callbacks import ReduceLROnPlateau , EarlyStopping
from keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

: 

In [ ]:
from tkinter import *
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from googletrans import Translator, LANGUAGES
from tkinter import Tk, Label, Button, filedialog
from PIL import Image, ImageTk

In [ ]:
#Reading dataset
# data = pd.read_csv('C:\\Users\\Vednant\\Documents\\f_year\\A_Z Handwritten Data.csv').astype('float32')
data = pd.read_csv('C:\\Users\\Aman Beast\\Downloads\\HCR code\\Dataset\\A_Z Handwritten Data.csv').astype('float32')
#astype() can be very helpful function in case data is not stored in correct format(data type).
print(data.head(10))

In [ ]:
#Split data into images and their labels.The '0' contains the labels we drop '0'column from X and use it in y to form label.
X = data.drop('0',axis = 1)
y = data['0']


In [ ]:
#Split data into traing set and testing set using sklearn library train_test_split().
train_x , test_x , train_y , test_y = train_test_split(X , y , test_size = 0.2)

#Reshape train and test image data so that can be displayed as an image.
#Initially in csv file present 784 columns of pixel data.that convert it to 28*28 pixels.
train_x = np.reshape(train_x.values , (train_x.shape[0] , 28 , 28))
test_x = np.reshape(test_x.values , (test_x.shape[0] , 28 , 28))

print('Train data shape: ' , train_x.shape)
print('Test data shape: ' , test_x.shape)

In [ ]:
#All labels in form of floating values.that convert to integer values. so we create dictionary to map the int values with char.
word_dict =  {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X', 24:'Y',25:'Z'}

In [ ]:
#Plotting the number of alphabets in the dataset.

y_int = np.int0(y)#y contain labels.convert labels into integer values
count = np.zeros(26, dtype='int')
for i in y_int:
    count[i] +=1

alphabets = []
for i in word_dict.values():
    alphabets.append(i)

fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.barh(alphabets, count)

plt.xlabel("Number of elements ")
plt.ylabel("Alphabets")
plt.grid()#use to add grid lines to plot.
plt.show()

In [ ]:
# Shuffle Data
# Shuffle dataset reduces the model to overfitting and genralizes well.
shuff = shuffle(train_x[:100])

fig, ax = plt.subplots(3,3, figsize = (10,10))
axes = ax.flatten()

for i in range(9):
    _, shu = cv2.threshold(shuff[i], 30, 200, cv2.THRESH_BINARY)
    axes[i].imshow(np.reshape(shuff[i], (28,28)), cmap="Greys")
plt.show()

In [ ]:
#Data Reshaping
#Reshaping the training and test dataset so that it can be put in the model.

train_X = train_x.reshape(train_x.shape[0],train_x.shape[1],train_x.shape[2],1)
print("New shape of train data: ", train_X.shape)

test_X = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2],1)
print("New shape of test data: ", test_X.shape)

#Here covert single float values to categorical values because CNN model takes input of labels and generate output as vector of probabilities.
# train_yOHE = to_categorical(train_y, num_classes = 26, dtype="int")
train_yOHE = to_categorical(train_y, num_classes = 26)
print("New shape of train labels: ", train_yOHE.shape)

# test_yOHE = to_categorical(test_y, num_classes = 26, dtype="int")
test_yOHE = to_categorical(test_y, num_classes = 26)
print("New shape of test labels: ", test_yOHE.shape)


In [ ]:
# Model Creation.
model = Sequential()#Because it is simple squential model.

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))#Adding Convolutional layer 
model.add(MaxPool2D(pool_size=(2, 2), strides=2))#Adding Maxpooling layer

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

#maxpool and covolutional layer are flattened into a vector of single dimension.
model.add(Flatten())

#Given the input to Dense Layer.
model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))

#Final layer has 26 units because dataset has 26 classes
#Since it is multiclass problem , the Softmax activation function is applied.
model.add(Dense(26,activation ="softmax"))


In [ ]:
#Compling and Fitting Model
#We are compling the model,where we define the optimizing and loss function to be used for fitting.
model.compile(optimizer = Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

#fit data into traing set
history = model.fit(train_X, train_yOHE, epochs=1,  validation_data = (test_X,test_yOHE))

In [ ]:
#Model Summary that tells us different layers defined in the model. and save model using model.save() function.
model.summary()
model.save(r'model_hand.h5')

In [ ]:
#Getting Train and Validation Accuracies and Losses
print('The validation accuracy is : ' , history.history['val_accuracy'])
print('The training accuracy is : ' , history.history['accuracy'])
print('The validation loss is : ' , history.history['val_loss'])
print('The training loss is : ' , history.history['loss'])

In [ ]:
#Doing Predictions on Test Data
fig, axes = plt.subplots(3,3, figsize=(8,9))
axes = axes.flatten()
for i,ax in enumerate(axes):
    img = np.reshape(test_X[i], (28,28))
    ax.imshow(img, cmap="Greys")
    
    pred = word_dict[np.argmax(test_yOHE[i])]
    ax.set_title("Prediction: "+pred)
    ax.grid()

In [ ]:
#----------- Main HCR Window -------------
win = Tk()
win.config(bg='wheat')
win.title('Handwritten Character Recognition')
# win.iconbitmap('icon 2.ico')
win.geometry('1400x780')
win.bind('<Escape>', lambda e, w=win: w.destroy())

#---------- Functions ------------

def open_image(frame):
    filepath = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.png")])
    print(filepath)
    img_file = ("r"+ "'" + filepath + "'")
    image = Image.open(filepath)
    image = image.resize((500, 500))

    photo = ImageTk.PhotoImage(image)

    label = Label(left_frame, image=photo)
    label.image = photo  # Keep a reference to prevent garbage collection
    label.place(x=80,y=50)

def exit():
    win.destroy()

def Translatebtn():
    win.destroy()
    root = Tk()
    root.geometry('1080x400')
    # root.iconbitmap('icon 2.ico')
    root.resizable(0, 0)
    root.title("HCR - Language Translator")
    # root.config(bg='#856ff8')
    root.config(bg='#0073e6')

    # Heading
    # Label(root, text="LANGUAGE TRANSLATOR", font="arial 20 bold", bg='#99e5f2').pack()
    Label(root, text="LANGUAGE TRANSLATOR", font="arial 20 bold", bg='#8babf1').pack()

    # INPUT AND OUTPUT TEXT WIDGET
    Label(root, text="Enter Text", font='arial 13 bold', bg='white smoke').place(x=200, y=60)
    Input_text = Text(root, font='arial 10', height=11, wrap=WORD, padx=5, pady=5, width=60)
    Input_text.place(x=30, y=100)

    Label(root, text="Translation", font='arial 13 bold', bg='white smoke').place(x=780, y=60)
    Output_text = Text(root, font='arial 10', height=11, wrap=WORD, padx=5, pady=5, width=60)
    Output_text.place(x=600, y=100)

    #------- Lang list -----------
    language = list(LANGUAGES.values())

    src_lang = ttk.Combobox(root, values=language, width=22)
    src_lang.place(x=20, y=60)
    src_lang.current(src_lang.set('english'))
    # src_lang.set('-Select input language-')

    dest_lang = ttk.Combobox(root, values=language, width=22)
    dest_lang.place(x=890, y=60)
    dest_lang.set('Select Output Language-')

    #---------  Define function ---------

    def Translate():
        translator = Translator()
        translated = translator.translate(text=Input_text.get(1.0, END), src=src_lang.get(), dest=dest_lang.get())

        Output_text.delete(1.0, END)
        Output_text.insert(END, translated.text)


    #---------  Translate Button ----------
    trans_btn = Button(root, text='Translate', font='arial 12 bold', pady=5, command=Translate, bg='royal blue1',
                       activebackground='sky blue')
    trans_btn.place(x=490, y=180)


# --------- Main Frames ------------
back_frame = Frame(win,bg='#5ba300',bd='5',relief=RIDGE)
back_frame.place(x=25,y=20,width=1480,height=760)

left_frame = Frame(win,bg='#0073e6',bd='5',relief=RIDGE)
# left_frame = Frame(win,bg='#856ff8',bd='5',relief=RIDGE)
left_frame.place(x=50,y=40,width=700,height=680)

l_title = Label(left_frame,text='Input Image',bg='yellow',font=('Times new roman',18))
l_title.grid(row=0,columnspan=2,pady=10,padx=10)

# separator = ttk.Separator(back_frame, orient='vertical')
# separator.place(relx=0.47, rely=0, relwidth=0.1, relheight=1)
select_button = Button(left_frame, text="Select Image",command=lambda: open_image(win))
select_button.place(x=300,y=590)

right_frame = Frame(win,bg='#856ff8',bd='5',relief=RIDGE)
right_frame.place(x=780,y=40,width=700,height=680)

r_title = Label(right_frame,text='Text Panel',bg='yellow',font=('Times new roman',18))
r_title.grid(row=0,columnspan=2,pady=10,padx=10)

Output_text = Text(right_frame, font='arial 20', height=18, wrap=WORD, padx=5, pady=5, width=42)
Output_text.place(x=20, y=50)

btn = Button(back_frame,text='Exit',font=('Times new roman',10,'bold'),bg='#4681f4',width='15',height='2',command=exit)
btn.place(x=1320,y=700)

btn = Button(back_frame,text='Traslate',font=('Times new roman',10,'bold'),bg='#4681f4',width='15',height='2',command=Translatebtn)
btn.place(x=1190,y=700)

# win.mainloop()

# #Doing the prediction on external images
# #img = cv2.imread(r'C:\Users\Vednant\Documents\my doc\x_img.jpg')
# img_path = r"C:\Users\Aman Beast\Desktop\HCR\A_output.jpg"
# img = cv2.imread(img_path)
# # img = cv2.imread(img_file)
# img_copy = img.copy()
# # img = cv2.imread(img_file)
# # if img is None:
# #     break
# img = img.copy()

# #converted from BGR representation(as OpenCV reads the image in BGR) to RGB for display , resized image to required dimension. 
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img = cv2.resize(img, (400,440))

In [ ]:
#Doing the prediction on external images
#img = cv2.imread(r'C:\Users\Vednant\Documents\my doc\x_img.jpg')
img_path = r"C:\Users\Aman Beast\Desktop\HCR\A_output.jpg"
img = cv2.imread(img_path)
# img = cv2.imread(img_file)
img_copy = img.copy()
# img = cv2.imread(img_file)
# if img is None:
#     break
img = img.copy()

#converted from BGR representation(as OpenCV reads the image in BGR) to RGB for display , resized image to required dimension. 
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (400,440))

In [ ]:
#Some Processing on the copied image(img_copy)
#Processing image mean removing gaussian noise , resize , reshape the image.
img_copy = cv2.GaussianBlur(img_copy, (7,7), 0)
img_gray = cv2.cvtColor(img_copy, cv2.COLOR_BGR2GRAY)
_, img_thresh = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY_INV)
img_final = cv2.resize(img_thresh, (28,28))
img_final =np.reshape(img_final, (1,28,28,1))

In [ ]:
#Doing prediction on processed image.
#Use np.argmax() function to get index of the class with highest predicted probability.using this exact character through dictioary
img_pred = word_dict[np.argmax(model.predict(img_final))]
print(img_pred)
cv2.putText(img, "Input Image", (20,25), cv2.FONT_HERSHEY_TRIPLEX, 0.7, color = (0,0,230))
cv2.putText(img, "Prediction: " + img_pred, (20,410), cv2.FONT_HERSHEY_DUPLEX, 1.3, color = (255,0,30))
cv2.imshow('Handwritten Character Recognition', img)

In [ ]:
#Setting up a waitkey in while loop
#waitKey() method allow user to display a window for given millisecond(1) or until any key is pressed(0).
#destroyAllWindows() simply destory all the windows we created

# while (1):
#     win.mainloop()
#     k = cv2.waitKey(1) & 0xFF #display a window 1 millisecond
#     if k == 27: # Press Escape key to exit(Stop) 
#         break
# cv2.destroyAllWindows()
